In [ ]:
# https://blog.sionic.ai/finetuning_llama

In [ ]:
# !nvidia-smi

Sat Nov 16 15:49:49 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.120                Driver Version: 550.120        CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               Off |   00000000:3B:00.0 Off |                  Off |
| 30%   34C    P0             68W /  230W |       4MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
import os
import torch
from datasets import load_dataset

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer


In [1]:
# pip install huggingface-hub

Note: you may need to restart the kernel to use updated packages.


In [15]:
# !huggingface-cli download mlabonne/Llama-3.1-70B-Instruct-lorablated-GGUF llama-3.1-70b-instruct-lorablated.Q4_K_M.gguf --local-dir /usr/share/d_ollama/.ollama/models/hf_model --local-dir-use-symlinks False

/home/mghan/sopjt/git/stackoverflow_src/venv_stackoverflow_src/lib/python3.10/site-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
/usr/share/d_ollama/.ollama/models/hf_model/llama-3.1-70b-instruct-lorablated.Q4_K_M.gguf


In [3]:
from huggingface_hub import interpreter_login

interpreter_login()

/home/mghan/sopjt/git/stackoverflow_src/venv_stackoverflow_src/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [9]:
# Hugging Face Basic Model 한국어 모델
base_model = "/usr/share/d_ollama/.ollama/models/hf_model"

# Custom Dataset ★ 본인이 hugging face 내 저장한 모델경로를 설정해야함 ★
hf_dataset = "myokyung/llama_test"

new_model = "Llama-hkcode-Ko-3-8B"

In [10]:
dataset = load_dataset(hf_dataset, split="train")

# 데이터 확인
print( dataset[28] )

{'text': '<s>[INST] Here is the question about software development. The title of the question is "How can I add my credentials into .gitignore but still execute my python?". and body of the question is "<p>I am confused on how to add my credentials to .gitignore when they are in my config file. I have database credentials stored in my config file and they are used to execute my code below:</p>\n<pre><code>import os \nimport json\nimport platform\nimport logging\nimport pymysql as pm\nimport boto3\n\nclass ClassName:\n    env=None\n    config=None\n\n    def __init__(self, env_filename):\n        self.env=env_filename\n        self.config=self.get_config()\n\n    def get_config(self):\n        with open(self.env) as file_in:\n            return json.load(file_in)\n        \n    def is_Windows():\n        if &quot;win&quot; in (platform.system().lower()):\n            return True\n        else:\n            return False\n\n    def DB_connection(self):\n        logger = logging.getLogger

5. 라마2모델 불러오기

In [12]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map={"": 0}
    # device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

OSError: /usr/share/d_ollama/.ollama/models/hf_model does not appear to have a file named config.json. Checkout 'https://huggingface.co//usr/share/d_ollama/.ollama/models/hf_model/tree/None' for available files.